In [1]:
import ctypes
import time
import sys

# Load the appropriate library depending on the operating system.
if sys.platform == 'win32':
    edsdk = ctypes.WinDLL('EDSDK.dll')
elif sys.platform == 'darwin':
    # Adjust the path if necessary; this assumes the EDSDK framework is installed.
    edsdk = ctypes.CDLL('EDSDK_v13.19.0_Macintosh/EDSDK/Framework/EDSDK.framework/EDSDK')
else:
    edsdk = ctypes.CDLL('libEDSDK.so')

# Define the function prototypes based on the documentation.
# For example, here’s how you might set up the EdsInitializeSDK function:

# The SDK functions typically return an error code (an integer), so set the return type accordingly.
edsdk.EdsInitializeSDK.argtypes = []  # No arguments
edsdk.EdsInitializeSDK.restype = ctypes.c_int

# Call initialization function.
status = edsdk.EdsInitializeSDK()
if status != 0:
    raise Exception("Failed to initialize the Canon EDSDK, error code: {}".format(status))
else:
    print("EDSDK successfully initialized")


CameraRef = ctypes.c_void_p

# Define the prototype for EdsOpenSession:
edsdk.EdsOpenSession.argtypes = [CameraRef]
edsdk.EdsOpenSession.restype = ctypes.c_int


def check(err):
    if err != 0:
        raise Exception("Error: 0x{:08X}".format(err))



EDSDK successfully initialized


In [2]:
try:
    # Initialize the EDSDK
    # check(edsdk.EdsInitializeSDK())
    # print("EDSDK initialized successfully.")

    # Get the camera list (this returns a pointer in cameraList)
    cameraList = ctypes.c_void_p()
    check(edsdk.EdsGetCameraList(ctypes.byref(cameraList)))
    print("Camera list obtained.")

    # Get the number of cameras connected
    camera_count = ctypes.c_uint(0)
    check(edsdk.EdsGetChildCount(cameraList, ctypes.byref(camera_count)))
    print("Number of cameras detected:", camera_count.value)

    if camera_count.value == 0:
        print("No cameras connected.")
    else:
        print("Camera is connected.")
        # Optionally, get the first camera (index 0)
        # For further operations like opening a session, you would use:
        # camera = ctypes.c_void_p()
        # check(edsdk.EdsGetChildAtIndex(cameraList, 0, ctypes.byref(camera)))
        # Then you might call EdsOpenSession(camera) and even EdsGetDeviceInfo(camera, ...)
        
finally:
    # Terminate the SDK to clean up resources
    try:
        edsdk.EdsTerminateSDK()
        print("EDSDK terminated.")
    except Exception:
        pass


Camera list obtained.
Number of cameras detected: 1
Camera is connected.
EDSDK terminated.


In [ ]:
kEdsCameraCommand_PressShutterButton = 0x00000000
kEdsCameraCommand_ShutterButton_Completely = 0x00000001
kEdsCameraCommand_ShutterButton_OFF = 0x00000000

kEdsFileCreateDisposition_CreateAlways = 2  # Create a new file (example)
kEdsAccess_ReadWrite = 2                    # Read/write access (example)

# Basic pointer types (using void* as a generic pointer)
CameraRef = ctypes.c_void_p
VolumeRef = ctypes.c_void_p
DirectoryItemRef = ctypes.c_void_p
StreamRef = ctypes.c_void_p

edsdk.EdsInitializeSDK.argtypes = []
edsdk.EdsInitializeSDK.restype = ctypes.c_int

edsdk.EdsTerminateSDK.argtypes = []
edsdk.EdsTerminateSDK.restype = ctypes.c_int

edsdk.EdsGetCameraList.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetCameraList.restype = ctypes.c_int

edsdk.EdsGetChildCount.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_uint)]
edsdk.EdsGetChildCount.restype = ctypes.c_int

edsdk.EdsGetChildAtIndex.argtypes = [ctypes.c_void_p, ctypes.c_uint, ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetChildAtIndex.restype = ctypes.c_int

edsdk.EdsOpenSession.argtypes = [CameraRef]
edsdk.EdsOpenSession.restype = ctypes.c_int

edsdk.EdsCloseSession.argtypes = [CameraRef]
edsdk.EdsCloseSession.restype = ctypes.c_int

edsdk.EdsSendCommand.argtypes = [CameraRef, ctypes.c_uint, ctypes.c_uint]
edsdk.EdsSendCommand.restype = ctypes.c_int

edsdk.EdsCreateFileStream.argtypes = [ctypes.c_char_p, ctypes.c_uint, ctypes.c_uint, ctypes.POINTER(StreamRef)]
edsdk.EdsCreateFileStream.restype = ctypes.c_int

edsdk.EdsDownload.argtypes = [DirectoryItemRef, ctypes.c_uint, StreamRef]
edsdk.EdsDownload.restype = ctypes.c_int

edsdk.EdsDownloadComplete.argtypes = [DirectoryItemRef]
edsdk.EdsDownloadComplete.restype = ctypes.c_int



In [ ]:
try:
    # Initialize EDSDK
    check(edsdk.EdsInitializeSDK())
    print("EDSDK initialized.")

    # Get camera list
    cameraList = ctypes.c_void_p()
    check(edsdk.EdsGetCameraList(ctypes.byref(cameraList)))

    # Get number of cameras connected
    camera_count = ctypes.c_uint(0)
    check(edsdk.EdsGetChildCount(cameraList, ctypes.byref(camera_count)))
    if camera_count.value == 0:
        raise Exception("No cameras found!")
    print("Found {} camera(s).".format(camera_count.value))

    # Get the first camera from the list
    camera = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(cameraList, 0, ctypes.byref(camera)))

    # Open a session with the camera
    check(edsdk.EdsOpenSession(camera))
    print("Session opened with camera.")
    # Trigger a capture:
    # 1. Press the shutter button completely.
    check(edsdk.EdsSendCommand(camera, kEdsCameraCommand_PressShutterButton, kEdsCameraCommand_ShutterButton_Completely))
    time.sleep(1)  # Wait for the exposure to complete
    # 2. Release the shutter button.
    check(edsdk.EdsSendCommand(camera, kEdsCameraCommand_PressShutterButton, kEdsCameraCommand_ShutterButton_OFF))
    print("Capture triggered.")

    # -----------------------------------------------------------------------------
    # 6. Download the captured image
    # -----------------------------------------------------------------------------
    # IMPORTANT: In a full implementation you must retrieve the directory item reference
    # for the captured image. This is typically done via event callbacks (e.g., on kEdsObjectEvent_DirItemCreated)
    # or by enumerating the camera’s storage. For demonstration, we assume you have obtained it:
    capturedItem = ctypes.c_void_p()  # Placeholder: replace with your actual obtained directory item

    # Check that capturedItem is valid (in real code, verify the reference is non-null)
    if not capturedItem:
        print("Warning: No captured image reference available. (Implement directory enumeration or event handling.)")
    else:
        # Define the save path (ensure it is a bytes object)
        save_path = b"./captured_image.jpg"
        stream = StreamRef()
        check(edsdk.EdsCreateFileStream(save_path, kEdsFileCreateDisposition_CreateAlways, kEdsAccess_ReadWrite, ctypes.byref(stream)))
        print("File stream created for:", save_path.decode())

        # For proper operation, you should query the file size from the directory item.
        # Here, we pass 0 as a placeholder.
        file_size = 0
        check(edsdk.EdsDownload(capturedItem, file_size, stream))
        check(edsdk.EdsDownloadComplete(capturedItem))
        print("Image downloaded to", save_path.decode())

    # Close the session and terminate the SDK
    check(edsdk.EdsCloseSession(camera))
    check(edsdk.EdsTerminateSDK())
    print("Session closed and SDK terminated.")

except Exception as e:
    print("An error occurred:", e)


In [2]:
import ctypes
import time
import sys

# -----------------------------------------------------------------------------
# Helper: Error checking
# -----------------------------------------------------------------------------
def check(err):
    if err != 0:
        raise Exception("Error: 0x{:08X}".format(err))

# -----------------------------------------------------------------------------
# 1. Load the EDSDK Library 
#    (Update the path below to match your environment)
# -----------------------------------------------------------------------------
if sys.platform == 'darwin':
    edsdk = ctypes.CDLL('/Users/jmitton/My Drive/SFU/CMPT 461/Project/WalmartRGB/EDSDK_v13.19.0_Macintosh/EDSDK/Framework/EDSDK.framework/EDSDK')
else:
    edsdk = ctypes.CDLL('EDSDK.dll')

# -----------------------------------------------------------------------------
# 2. Define basic types and constants (adjust these to your SDK's header definitions)
# -----------------------------------------------------------------------------
# Pointer types
CameraRef = ctypes.c_void_p
DirectoryItemRef = ctypes.c_void_p
StreamRef = ctypes.c_void_p

# Shutter command constants (values are examples; confirm with EDSDK.h)
kEdsCameraCommand_PressShutterButton = 0x00000000
kEdsCameraCommand_ShutterButton_Completely = 0x00000001
kEdsCameraCommand_ShutterButton_OFF = 0x00000000

# File stream constants (example values; see EDSDK documentation)
kEdsFileCreateDisposition_CreateAlways = 2  # Create new file always
kEdsAccess_ReadWrite = 2                    # Read/write access

# -----------------------------------------------------------------------------
# 3. Define function prototypes (only the functions used in this example)
# -----------------------------------------------------------------------------

# Initialize and terminate SDK
edsdk.EdsInitializeSDK.argtypes = []
edsdk.EdsInitializeSDK.restype = ctypes.c_int

edsdk.EdsTerminateSDK.argtypes = []
edsdk.EdsTerminateSDK.restype = ctypes.c_int

# Get camera list and count
edsdk.EdsGetCameraList.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetCameraList.restype = ctypes.c_int

edsdk.EdsGetChildCount.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_uint)]
edsdk.EdsGetChildCount.restype = ctypes.c_int

edsdk.EdsGetChildAtIndex.argtypes = [ctypes.c_void_p, ctypes.c_uint, ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetChildAtIndex.restype = ctypes.c_int

# Open and close session with camera
edsdk.EdsOpenSession.argtypes = [CameraRef]
edsdk.EdsOpenSession.restype = ctypes.c_int

edsdk.EdsCloseSession.argtypes = [CameraRef]
edsdk.EdsCloseSession.restype = ctypes.c_int

# Send a command to the camera (for shutter control)
edsdk.EdsSendCommand.argtypes = [CameraRef, ctypes.c_uint, ctypes.c_uint]
edsdk.EdsSendCommand.restype = ctypes.c_int

# Create a file stream for downloading images
edsdk.EdsCreateFileStream.argtypes = [ctypes.c_char_p, ctypes.c_uint, ctypes.c_uint, ctypes.POINTER(StreamRef)]
edsdk.EdsCreateFileStream.restype = ctypes.c_int

# Download and complete download operations
edsdk.EdsDownload.argtypes = [DirectoryItemRef, ctypes.c_uint, StreamRef]
edsdk.EdsDownload.restype = ctypes.c_int

edsdk.EdsDownloadComplete.argtypes = [DirectoryItemRef]
edsdk.EdsDownloadComplete.restype = ctypes.c_int

# -----------------------------------------------------------------------------
# 4. Main workflow: Capture image and download it
# -----------------------------------------------------------------------------
try:
    # Initialize EDSDK
    check(edsdk.EdsInitializeSDK())
    print("EDSDK initialized.")

    # Get camera list
    cameraList = ctypes.c_void_p()
    check(edsdk.EdsGetCameraList(ctypes.byref(cameraList)))
    
    # Get number of cameras
    camera_count = ctypes.c_uint(0)
    check(edsdk.EdsGetChildCount(cameraList, ctypes.byref(camera_count)))
    if camera_count.value == 0:
        raise Exception("No cameras found!")

    # Get the first camera (index 0)
    camera = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(cameraList, 0, ctypes.byref(camera)))
    print("Camera obtained.")

    # Open a session with the camera
    check(edsdk.EdsOpenSession(camera))
    print("Session opened with camera.")

    # Trigger a capture:
    # a) Press the shutter button completely
    check(edsdk.EdsSendCommand(camera, kEdsCameraCommand_PressShutterButton, kEdsCameraCommand_ShutterButton_Completely))
    time.sleep(1)  # Wait for exposure time; adjust delay as needed
    
    # b) Release the shutter button
    check(edsdk.EdsSendCommand(camera, kEdsCameraCommand_PressShutterButton, kEdsCameraCommand_ShutterButton_OFF))
    print("Capture triggered.")

    # -----------------------------------------------------------------------------
    # 5. Handling the captured image
    # -----------------------------------------------------------------------------
    # In a full implementation, you should register a callback to receive the 
    # kEdsObjectEvent_DirItemCreated event which provides a reference to the captured image.
    #
    # For this example, we assume you have obtained the directory item reference (capturedItem).
    capturedItem = ctypes.c_void_p()  # This should be replaced with the actual directory item obtained!
    
    if not capturedItem:
        # If you haven't implemented event handling, you could alternatively poll the camera storage
        # to find the newest file. This placeholder warns you that additional steps are needed.
        print("Captured image reference not obtained. You need to implement event handling or polling to retrieve the new file reference.")
    else:
        # Create a file stream to save the image locally
        save_path = b"./captured_image.jpg"
        stream = StreamRef()
        check(edsdk.EdsCreateFileStream(save_path, kEdsFileCreateDisposition_CreateAlways, kEdsAccess_ReadWrite, ctypes.byref(stream)))
        print("File stream created for:", save_path.decode())

        # NOTE: In a complete implementation, you would query the file size for the download from the directory item.
        file_size = 0  # Use the proper file size here
        check(edsdk.EdsDownload(capturedItem, file_size, stream))
        check(edsdk.EdsDownloadComplete(capturedItem))
        print("Image downloaded to", save_path.decode())

    # Close the session
    check(edsdk.EdsCloseSession(camera))
    print("Session closed.")

finally:
    # Terminate the SDK to clean up
    try:
        check(edsdk.EdsTerminateSDK())
        print("EDSDK terminated.")
    except Exception:
        pass

# except Exception as e:
#     print("An error occurred:", e)


EDSDK initialized.
Camera obtained.
Session opened with camera.
Capture triggered.
Captured image reference not obtained. You need to implement event handling or polling to retrieve the new file reference.
Session closed.
EDSDK terminated.


In [4]:
import os


working_directory = os.getcwd()
filename = "captured_image.jpg"  # Change this if you want a unique name each time.
full_save_path = os.path.join(working_directory, filename)
# The EDSDK functions expect a byte-string:
save_path_bytes = full_save_path.encode('utf-8')

print("Saving image to:", full_save_path)

# Create a file stream pointing to the desired file.
stream = ctypes.c_void_p()  # This is our StreamRef
err = edsdk.EdsCreateFileStream(save_path_bytes,
                                kEdsFileCreateDisposition_CreateAlways,
                                kEdsAccess_ReadWrite,
                                ctypes.byref(stream))
check(err)
print("File stream created.")

# At this point, capturedItem should be a valid DirectoryItemRef for the captured image.
# In a full application, this is obtained via an event callback such as kEdsObjectEvent_DirItemCreated.
# For this example, we assume that capturedItem is valid:
capturedItem = ctypes.c_void_p()  # Replace with your actual reference

# (Optional) Obtain the file size from the directory item info.
# Typically, you would use EdsGetDirectoryItemInfo() to fill a structure that includes file size.
# For demonstration, we use a placeholder; ensure you pass the correct size.
file_size = 0  # Replace 0 with the actual file size from the camera.

# Begin the download from the camera to the file stream.
err = edsdk.EdsDownload(capturedItem, file_size, stream)
check(err)
print("Download initiated.")

# Signal that the download is complete.
err = edsdk.EdsDownloadComplete(capturedItem)
check(err)
print("Download complete. Image saved in the working directory.")


Saving image to: /Users/jmitton/My Drive/SFU/CMPT 461/Project/WalmartRGB/captured_image.jpg


Exception: Error: 0x00000002

In [1]:
import ctypes
import os
import time
import sys

def check(err):
    if err != 0:
        raise Exception("Error: 0x{:08X}".format(err))

# -----------------------------------------------------------------------------
# 1. Load the EDSDK Library (update the path as necessary)
# -----------------------------------------------------------------------------
if sys.platform == 'darwin':
    edsdk = ctypes.CDLL('/Users/jmitton/My Drive/SFU/CMPT 461/Project/WalmartRGB/EDSDK_v13.19.0_Macintosh/EDSDK/Framework/EDSDK.framework/EDSDK')
else:
    edsdk = ctypes.CDLL('EDSDK.dll')

# -----------------------------------------------------------------------------
# 2. Define Basic Types and Constants
# -----------------------------------------------------------------------------
CameraRef = ctypes.c_void_p
DirectoryItemRef = ctypes.c_void_p
StreamRef = ctypes.c_void_p

# Shutter Command Constants (verify these with your EDSDK headers)
kEdsCameraCommand_PressShutterButton      = 0x00000000
kEdsCameraCommand_ShutterButton_Completely  = 0x00000001
kEdsCameraCommand_ShutterButton_OFF         = 0x00000000

# File stream constants (verify in the SDK documentation)
kEdsFileCreateDisposition_CreateAlways = 2
kEdsAccess_ReadWrite = 2

# -----------------------------------------------------------------------------
# 3. Define Function Prototypes
# -----------------------------------------------------------------------------
edsdk.EdsInitializeSDK.argtypes = []
edsdk.EdsInitializeSDK.restype = ctypes.c_int

edsdk.EdsTerminateSDK.argtypes = []
edsdk.EdsTerminateSDK.restype = ctypes.c_int

edsdk.EdsGetCameraList.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetCameraList.restype = ctypes.c_int

edsdk.EdsGetChildCount.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_uint)]
edsdk.EdsGetChildCount.restype = ctypes.c_int

edsdk.EdsGetChildAtIndex.argtypes = [ctypes.c_void_p, ctypes.c_uint, ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetChildAtIndex.restype = ctypes.c_int

edsdk.EdsOpenSession.argtypes = [CameraRef]
edsdk.EdsOpenSession.restype = ctypes.c_int

edsdk.EdsCloseSession.argtypes = [CameraRef]
edsdk.EdsCloseSession.restype = ctypes.c_int

edsdk.EdsSendCommand.argtypes = [CameraRef, ctypes.c_uint, ctypes.c_uint]
edsdk.EdsSendCommand.restype = ctypes.c_int

edsdk.EdsCreateFileStream.argtypes = [ctypes.c_char_p, ctypes.c_uint, ctypes.c_uint, ctypes.POINTER(StreamRef)]
edsdk.EdsCreateFileStream.restype = ctypes.c_int

edsdk.EdsDownload.argtypes = [DirectoryItemRef, ctypes.c_uint, StreamRef]
edsdk.EdsDownload.restype = ctypes.c_int

edsdk.EdsDownloadComplete.argtypes = [DirectoryItemRef]
edsdk.EdsDownloadComplete.restype = ctypes.c_int

# -----------------------------------------------------------------------------
# 4. Main Capture Process (Using Folder at Index 0)
# -----------------------------------------------------------------------------
try:
    # Initialize the SDK.
    check(edsdk.EdsInitializeSDK())
    print("EDSDK initialized.")

    # Retrieve the camera list.
    camList = ctypes.c_void_p()
    check(edsdk.EdsGetCameraList(ctypes.byref(camList)))
    
    # Get camera count.
    camCount = ctypes.c_uint(0)
    check(edsdk.EdsGetChildCount(camList, ctypes.byref(camCount)))
    if camCount.value == 0:
        raise Exception("No cameras found!")
    print("Found {} camera(s).".format(camCount.value))

    # Get the first camera.
    camera = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(camList, 0, ctypes.byref(camera)))
    print("Camera obtained.")

    # Open a session with the camera.
    check(edsdk.EdsOpenSession(camera))
    print("Session opened with camera.")

    # Obtain the SD card volume.
    volume = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(camera, 0, ctypes.byref(volume)))
    print("Volume (SD card) obtained.")

    # Directly get the folder at index 0.
    dcim_folder = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(volume, 0, ctypes.byref(dcim_folder)))
    print("DCIM obtained.")

    canon100_folder = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(dcim_folder, 0, ctypes.byref(canon100_folder)))
    print("canon100 folder obtained.")

    # Record baseline file count in folder.
    baseline_count = ctypes.c_uint(0)
    check(edsdk.EdsGetChildCount(canon100_folder, ctypes.byref(baseline_count)))
    print("Initial file count in folder:", baseline_count.value)

    # Trigger a capture on the camera.
    check(edsdk.EdsSendCommand(camera, kEdsCameraCommand_PressShutterButton, kEdsCameraCommand_ShutterButton_Completely))
    time.sleep(1)  # Allow exposure time.
    check(edsdk.EdsSendCommand(camera, kEdsCameraCommand_PressShutterButton, kEdsCameraCommand_ShutterButton_OFF))
    print("Capture triggered.")

    # Poll the folder for a new file.
    timeout = 5.0  # Increase timeout if needed.

    time.sleep(timeout)

    new_count = ctypes.c_uint(0)
    while timeout > 0:
        check(edsdk.EdsGetChildCount(canon100_folder, ctypes.byref(new_count)))
        if new_count.value > baseline_count.value:
            break
        time.sleep(0.5)
        timeout -= 0.5

    if new_count.value <= baseline_count.value:
        raise Exception("No new file detected after capture within the timeout period!")
    else:
        print("New file detected. File count increased from {} to {}.".format(baseline_count.value, new_count.value))
        # Assume the new file is at the last index of the folder.
        new_index = new_count.value - 1
        capturedItem = ctypes.c_void_p()
        check(edsdk.EdsGetChildAtIndex(dcim_folder, new_index, ctypes.byref(capturedItem)))
        print("Captured image reference obtained from folder at index 0, position {}.".format(new_index))

    # Download the captured image.
    working_dir = os.getcwd()
    filename = "captured_image.jpg"  # Use a timestamp for uniqueness if needed.
    file_path = os.path.join(working_dir, filename)
    file_path_bytes = file_path.encode('utf-8')
    print("Saving image to:", file_path)
    
    stream = ctypes.c_void_p()
    check(edsdk.EdsCreateFileStream(file_path_bytes,
                                    kEdsFileCreateDisposition_CreateAlways,
                                    kEdsAccess_ReadWrite,
                                    ctypes.byref(stream)))
    print("File stream created.")

    # For demonstration, we use 0 as a placeholder for file size.
    file_size = 0
    check(edsdk.EdsDownload(capturedItem, file_size, stream))
    check(edsdk.EdsDownloadComplete(capturedItem))
    print("Image downloaded successfully and saved to", file_path)

    # Close the session.
    check(edsdk.EdsCloseSession(camera))
    print("Session closed.")

finally:
    try:
        check(edsdk.EdsTerminateSDK())
        print("EDSDK terminated.")
    except Exception:
        pass

# except Exception as e:
#     print("An error occurred:", e)


EDSDK initialized.
Found 1 camera(s).
Camera obtained.
Session opened with camera.
Volume (SD card) obtained.
DCIM obtained.
canon100 folder obtained.
Initial file count in folder: 12
Capture triggered.
EDSDK terminated.


Exception: No new file detected after capture within the timeout period!

In [1]:
import ctypes
import os
import time
import sys

def check(err):
    if err != 0:
        raise Exception("Error: 0x{:08X}".format(err))

# -----------------------------------------------------------------------------
# 1. Load the EDSDK Library (update the path as necessary)
# -----------------------------------------------------------------------------
if sys.platform == 'darwin':
    edsdk = ctypes.CDLL('/Users/jmitton/My Drive/SFU/CMPT 461/Project/WalmartRGB/EDSDK_v13.19.0_Macintosh/EDSDK/Framework/EDSDK.framework/EDSDK')
else:
    edsdk = ctypes.CDLL('EDSDK.dll')

# -----------------------------------------------------------------------------
# 2. Define Basic Types and Constants
# -----------------------------------------------------------------------------
CameraRef = ctypes.c_void_p
DirectoryItemRef = ctypes.c_void_p
StreamRef = ctypes.c_void_p

# Shutter Command Constants (verify these with your EDSDK headers)
kEdsCameraCommand_PressShutterButton      = 0x00000000
kEdsCameraCommand_ShutterButton_Completely  = 0x00000001
kEdsCameraCommand_ShutterButton_OFF         = 0x00000000

# File stream constants (verify in the SDK documentation)
kEdsFileCreateDisposition_CreateAlways = 2
kEdsAccess_ReadWrite = 2

# --- Tethered Shooting: Save Destination Property ---
# Property ID for SaveTo is defined in the SDK header (usually 0x0000000B)
kEdsPropID_SaveTo = 0x0000000B  
# Destination values (verify with your header)
kEdsSaveTo_Card = 0  # Save images to the SD card only
kEdsSaveTo_PC   = 1  # Save images to the PC only
kEdsSaveTo_Both = 2  # Save images to both the SD card and PC

# -----------------------------------------------------------------------------
# 3. Define Function Prototypes
# -----------------------------------------------------------------------------
edsdk.EdsInitializeSDK.argtypes = []
edsdk.EdsInitializeSDK.restype = ctypes.c_int

edsdk.EdsTerminateSDK.argtypes = []
edsdk.EdsTerminateSDK.restype = ctypes.c_int

edsdk.EdsGetCameraList.argtypes = [ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetCameraList.restype = ctypes.c_int

edsdk.EdsGetChildCount.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_uint)]
edsdk.EdsGetChildCount.restype = ctypes.c_int

edsdk.EdsGetChildAtIndex.argtypes = [ctypes.c_void_p, ctypes.c_uint, ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetChildAtIndex.restype = ctypes.c_int

edsdk.EdsOpenSession.argtypes = [CameraRef]
edsdk.EdsOpenSession.restype = ctypes.c_int

edsdk.EdsCloseSession.argtypes = [CameraRef]
edsdk.EdsCloseSession.restype = ctypes.c_int

edsdk.EdsSendCommand.argtypes = [CameraRef, ctypes.c_uint, ctypes.c_uint]
edsdk.EdsSendCommand.restype = ctypes.c_int

edsdk.EdsSetPropertyData.argtypes = [CameraRef, ctypes.c_uint, ctypes.c_uint, ctypes.c_uint, ctypes.c_void_p]
edsdk.EdsSetPropertyData.restype = ctypes.c_int

edsdk.EdsCreateFileStream.argtypes = [ctypes.c_char_p, ctypes.c_uint, ctypes.c_uint, ctypes.POINTER(StreamRef)]
edsdk.EdsCreateFileStream.restype = ctypes.c_int

edsdk.EdsDownload.argtypes = [DirectoryItemRef, ctypes.c_uint, StreamRef]
edsdk.EdsDownload.restype = ctypes.c_int

edsdk.EdsDownloadComplete.argtypes = [DirectoryItemRef]
edsdk.EdsDownloadComplete.restype = ctypes.c_int

# -----------------------------------------------------------------------------
# 4. Main Capture Process (Using Folder at Index 0)
# -----------------------------------------------------------------------------
try:
    # Initialize the SDK.
    check(edsdk.EdsInitializeSDK())
    print("EDSDK initialized.")

    # Retrieve the camera list.
    camList = ctypes.c_void_p()
    check(edsdk.EdsGetCameraList(ctypes.byref(camList)))
    
    # Get camera count.
    camCount = ctypes.c_uint(0)
    check(edsdk.EdsGetChildCount(camList, ctypes.byref(camCount)))
    if camCount.value == 0:
        raise Exception("No cameras found!")
    print("Found {} camera(s).".format(camCount.value))

    # Get the first camera.
    camera = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(camList, 0, ctypes.byref(camera)))
    print("Camera obtained.")

    # Open a session with the camera.
    check(edsdk.EdsOpenSession(camera))
    print("Session opened with camera.")

    # --- Set Tethered Shooting Mode ---
    # Set the capture destination to Both, so the image is saved on the SD card and sent to the PC.
    destination = ctypes.c_uint(kEdsSaveTo_Both)
    check(edsdk.EdsSetPropertyData(camera, kEdsPropID_SaveTo, 0, ctypes.sizeof(destination), ctypes.byref(destination)))
    print("Capture destination set to Both (tethered shooting mode).")

    # Obtain the SD card volume (assumed to be the first child of the camera).
    volume = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(camera, 0, ctypes.byref(volume)))
    print("Volume (SD card) obtained.")

    # Directly get the folder at index 0 (assumed to be the DCIM folder, e.g., "100CANON").
    dcim_folder = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(volume, 0, ctypes.byref(dcim_folder)))
    print("DCIM folder obtained.")

    # Optionally, if your folder structure is deeper (e.g., DCIM/100CANON), retrieve the child folder:
    canon_folder = ctypes.c_void_p()
    check(edsdk.EdsGetChildAtIndex(dcim_folder, 0, ctypes.byref(canon_folder)))
    print("Target folder (assumed '100CANON') obtained.")

    # Record baseline file count in the target folder.
    baseline_count = ctypes.c_uint(0)
    check(edsdk.EdsGetChildCount(canon_folder, ctypes.byref(baseline_count)))
    print("Initial file count in folder:", baseline_count.value)

    time.sleep(2)

    # Trigger a capture on the camera.
    check(edsdk.EdsSendCommand(camera, kEdsCameraCommand_PressShutterButton, kEdsCameraCommand_ShutterButton_Completely))
    time.sleep(1)  # Allow exposure time.
    check(edsdk.EdsSendCommand(camera, kEdsCameraCommand_PressShutterButton, kEdsCameraCommand_ShutterButton_OFF))
    print("Capture triggered.")

    # Poll the folder for a new file.
    timeout = 20.0  # Increase timeout if needed.
    new_count = ctypes.c_uint(0)
    while timeout > 0:
        check(edsdk.EdsGetChildCount(canon_folder, ctypes.byref(new_count)))
        if new_count.value > baseline_count.value:
            break
        time.sleep(0.5)
        timeout -= 0.5

    if new_count.value <= baseline_count.value:
        raise Exception("No new file detected after capture within the timeout period!")
    else:
        print("New file detected. File count increased from {} to {}.".format(baseline_count.value, new_count.value))
        # Assume the new file is at the last index.
        new_index = new_count.value - 1
        capturedItem = ctypes.c_void_p()
        check(edsdk.EdsGetChildAtIndex(canon_folder, new_index, ctypes.byref(capturedItem)))
        print("Captured image reference obtained from folder at index 0, position {}.".format(new_index))

    # Download the captured image.
    working_dir = os.getcwd()
    filename = "captured_image.jpg"  # Use a timestamp for uniqueness if needed.
    file_path = os.path.join(working_dir, filename)
    file_path_bytes = file_path.encode('utf-8')
    print("Saving image to:", file_path)
    
    stream = ctypes.c_void_p()
    check(edsdk.EdsCreateFileStream(file_path_bytes,
                                    kEdsFileCreateDisposition_CreateAlways,
                                    kEdsAccess_ReadWrite,
                                    ctypes.byref(stream)))
    print("File stream created.")

    # For demonstration, we use 0 as a placeholder for file size.
    # In production, query the actual file size via EdsGetDirectoryItemInfo.
    file_size = 0
    check(edsdk.EdsDownload(capturedItem, file_size, stream))
    check(edsdk.EdsDownloadComplete(capturedItem))
    print("Image downloaded successfully and saved to", file_path)

    # Close the session.
    check(edsdk.EdsCloseSession(camera))
    print("Session closed.")

finally:
    try:
        check(edsdk.EdsTerminateSDK())
        print("EDSDK terminated.")
    except Exception:
        pass

# except Exception as e:
#     print("An error occurred:", e)


EDSDK initialized.
Found 1 camera(s).
Camera obtained.
Session opened with camera.
Capture destination set to Both (tethered shooting mode).
Volume (SD card) obtained.
DCIM folder obtained.
Target folder (assumed '100CANON') obtained.
Initial file count in folder: 14
EDSDK terminated.


Exception: Error: 0x00008D07

In [1]:
import ctypes
import time
import sys


# Load the EDSDK dynamic library.
# Adjust the library name/path depending on your operating system.
# Load the appropriate library depending on the operating system.
if sys.platform == 'win32':
    edsdk = ctypes.WinDLL('EDSDK.dll')
elif sys.platform == 'darwin':
    # Adjust the path if necessary; this assumes the EDSDK framework is installed.
    edsdk = ctypes.CDLL('EDSDK_v13.19.0_Macintosh/EDSDK/Framework/EDSDK.framework/EDSDK')
else:
    edsdk = ctypes.CDLL('libEDSDK.so')


# Define common types.
EdsError = ctypes.c_uint32
EdsCameraRef = ctypes.c_void_p       # Used for camera objects, volumes, folders, etc.
EdsCameraListRef = ctypes.c_void_p
EdsStreamRef = ctypes.c_void_p

# Constants (you may need to verify these values in the SDK header files).
kEdsCameraCommand_TakePicture = 0x00000000
kEdsPropID_SaveTo = 0x0000000b            # As defined in EDSDKTypes.h
kEdsSaveTo_Host = 1                       # Typically, 1 indicates the host-only mode.
kEdsFileCreateDisposition_CreateAlways = 1
kEdsAccess_ReadWrite = 2

# --- Function prototypes

# Initialization and termination.
edsdk.EdsInitializeSDK.restype = EdsError
edsdk.EdsTerminateSDK.restype = EdsError

# Camera list and selection.
edsdk.EdsGetCameraList.restype = EdsError
edsdk.EdsGetCameraList.argtypes = [ctypes.POINTER(EdsCameraListRef)]
edsdk.EdsGetChildCount.restype = EdsError
edsdk.EdsGetChildCount.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_uint32)]
edsdk.EdsGetChildAtIndex.restype = EdsError
edsdk.EdsGetChildAtIndex.argtypes = [ctypes.c_void_p, ctypes.c_int, ctypes.POINTER(EdsCameraRef)]

# Session control.
edsdk.EdsOpenSession.restype = EdsError
edsdk.EdsOpenSession.argtypes = [EdsCameraRef]
edsdk.EdsCloseSession.restype = EdsError
edsdk.EdsCloseSession.argtypes = [EdsCameraRef]

# Camera commands.
edsdk.EdsSendCommand.restype = EdsError
edsdk.EdsSendCommand.argtypes = [EdsCameraRef, ctypes.c_uint32, ctypes.c_uint32]

# Property access.
edsdk.EdsSetPropertyData.restype = EdsError
edsdk.EdsSetPropertyData.argtypes = [EdsCameraRef, ctypes.c_uint32, ctypes.c_int, ctypes.c_uint32, ctypes.c_void_p]

# Object release.
edsdk.EdsRelease.restype = ctypes.c_uint32
edsdk.EdsRelease.argtypes = [ctypes.c_void_p]

# Memory stream creation.
edsdk.EdsCreateMemoryStream.restype = EdsError
edsdk.EdsCreateMemoryStream.argtypes = [ctypes.c_uint32, ctypes.POINTER(EdsStreamRef)]

# Download functions.
edsdk.EdsDownload.restype = EdsError
edsdk.EdsDownload.argtypes = [ctypes.c_void_p, ctypes.c_uint32, EdsStreamRef]
edsdk.EdsDownloadComplete.restype = EdsError
edsdk.EdsDownloadComplete.argtypes = [ctypes.c_void_p]

# For getting a pointer to the memory in the stream.
edsdk.EdsGetPointer.restype = EdsError
edsdk.EdsGetPointer.argtypes = [EdsStreamRef, ctypes.POINTER(ctypes.c_void_p)]
edsdk.EdsGetLength.restype = EdsError
edsdk.EdsGetLength.argtypes = [EdsStreamRef, ctypes.POINTER(ctypes.c_uint32)]

# --- Error checking helper.
def check_error(err):
    if err != 0:  # 0 means EDS_ERR_OK.
        raise Exception(f"EDSDK error: {hex(err)}")

# --- Example function: Capture image and download into a memory stream.
def capture_image_to_memory():
    # 1. Initialize the SDK.
    err = edsdk.EdsInitializeSDK()
    check_error(err)
    print("SDK Initialized.")

    try:
        # 2. Get the camera list.
        camera_list = EdsCameraListRef()
        err = edsdk.EdsGetCameraList(ctypes.byref(camera_list))
        check_error(err)
        print("Camera list obtained.")

        # 3. Ensure at least one camera is connected.
        count = ctypes.c_uint32(0)
        err = edsdk.EdsGetChildCount(camera_list, ctypes.byref(count))
        check_error(err)
        print(f"Number of cameras found: {count.value}")
        if count.value == 0:
            raise Exception("No cameras detected.")

        # 4. Get a reference to the first camera.
        camera = EdsCameraRef()
        err = edsdk.EdsGetChildAtIndex(camera_list, 0, ctypes.byref(camera))
        check_error(err)
        print("Camera reference acquired.")

        # 5. Open a session with the camera.
        err = edsdk.EdsOpenSession(camera)
        check_error(err)
        print("Camera session opened.")

        # 6. Set the camera's save destination to the host (memory only, not to the SD card).
        host_mode = ctypes.c_uint32(kEdsSaveTo_Host)
        err = edsdk.EdsSetPropertyData(camera, kEdsPropID_SaveTo, 0,
                                        ctypes.sizeof(host_mode), ctypes.byref(host_mode))
        check_error(err)
        print("Camera set to save images to the host.")

        # 7. Trigger the capture command.
        err = edsdk.EdsSendCommand(camera, kEdsCameraCommand_TakePicture, 0)
        check_error(err)
        print("Capture command sent; camera is taking a picture.")

        # Wait for the image to be ready. In robust applications use an event callback (kEdsObjectEvent_DirItemCreated).
        time.sleep(5)

        # 8. At this point, because the camera is set to host-only saving, the SDK should create an image object
        #    that you can download directly without accessing the SD card structure.
        #    (The method to obtain the file reference may differ by model; some SDK samples show that you can
        #    register an event handler that returns the file object pointer.)
        #
        # For illustration, assume you have a valid file object reference "file_ref" and its size "file_size".
        # In actual implementations, the file_ref is provided via a callback upon the event notification.
        #
        # For this example, we assume you already have file_ref and file_size.
        # Replace these with the actual code required to get the file object from the event.
        file_ref = ctypes.c_void_p()   # Dummy placeholder; acquire this from the object event in practice.
        file_size = ctypes.c_uint32(0)   # Dummy placeholder.

        # ----
        # To simulate the download, let’s assume you already obtained file_ref and file_size.
        # The following block demonstrates how to create a memory stream and download the image into it.
        # ----

        # 9. Create a memory stream with a predefined buffer size.
        mem_stream = EdsStreamRef()
        mem_buffer_size = 10 * 1024 * 1024  # 10 MB buffer (adjust as needed)
        err = edsdk.EdsCreateMemoryStream(mem_buffer_size, ctypes.byref(mem_stream))
        check_error(err)
        print("Memory stream created for image download.")

        # 10. Download the image directly into the memory stream.
        err = edsdk.EdsDownload(file_ref, file_size.value, mem_stream)
        check_error(err)
        err = edsdk.EdsDownloadComplete(file_ref)
        check_error(err)
        print("Image downloaded into memory.")

        # 11. Retrieve the pointer and length to access the image data.
        data_ptr = ctypes.c_void_p()
        err = edsdk.EdsGetPointer(mem_stream, ctypes.byref(data_ptr))
        check_error(err)
        actual_length = ctypes.c_uint32(0)
        err = edsdk.EdsGetLength(mem_stream, ctypes.byref(actual_length))
        check_error(err)
        print(f"Downloaded image is {actual_length.value} bytes in memory.")

        # Now you can, for example, convert the memory block to a Python bytes object.
        buffer = (ctypes.c_char * actual_length.value).from_address(data_ptr.value)
        image_data = bytes(buffer)
        print("Image data is ready in memory and can be processed as needed.")

        # Clean up the file object and streams as required.
        edsdk.EdsRelease(mem_stream)
        edsdk.EdsRelease(file_ref)  # Release your file object obtained from the event.
        edsdk.EdsRelease(camera)
        edsdk.EdsRelease(camera_list)
        print("All camera objects released.")

    finally:
        # 12. Terminate the SDK.
        err = edsdk.EdsTerminateSDK()
        check_error(err)
        print("SDK terminated.")

if __name__ == "__main__":
    try:
        capture_image_to_memory()
    except Exception as e:
        print("Error:", e)


SDK Initialized.
Camera list obtained.
Number of cameras found: 1
Camera reference acquired.
Camera session opened.
Camera set to save images to the host.
Capture command sent; camera is taking a picture.
Memory stream created for image download.
SDK terminated.
Error: EDSDK error: 0x61


In [ ]:
import ctypes
import time
import sys

# --- Type Definitions and Constants ---
EdsError = ctypes.c_uint32           # Error type (0 means EDS_ERR_OK)
EdsCameraRef = ctypes.c_void_p       # Used for camera, volume, folder, file objects
EdsCameraListRef = ctypes.c_void_p
EdsStreamRef = ctypes.c_void_p

EDS_MAX_NAME = 256

# Property and command constants (verify these in your header files)
kEdsPropID_SaveTo = 0x0000000b        # Save destination property
kEdsSaveTo_Host = 1                   # Host-only mode (do not write to SD card)
kEdsCameraCommand_TakePicture = 0x00000000

# Event constants
kEdsObjectEvent_DirItemCreated = 0x00000102
# Use an event mask that registers for all object events.
kEdsObjectEvent_All = 0xFFFFFFFF

# --- Structure for Directory Item Info ---
class EdsDirectoryItemInfo(ctypes.Structure):
    _fields_ = [
        ("size", ctypes.c_uint32),                      # File size in bytes.
        ("isFolder", ctypes.c_uint32),                  # Nonzero if item is a folder.
        ("szFileName", ctypes.c_char * EDS_MAX_NAME)      # File name as a C-string.
        # Additional fields exist in the full SDK definition.
    ]

# --- Global Variable to Hold the Received File Handle ---
captured_file_ref = None

# --- Callback Function for Object Events ---
# The expected signature is: (event, ref, param, context)
CALLBACK_FUNC = ctypes.CFUNCTYPE(EdsError, ctypes.c_int, ctypes.c_void_p, ctypes.c_void_p, ctypes.c_void_p)

def object_event_handler(event, ref, param, context):
    global captured_file_ref
    # Debug: print any received event
    print(f"Callback: Received event 0x{event:08x}")
    if event == kEdsObjectEvent_DirItemCreated:
        captured_file_ref = param  # Store the valid file object handle from 'param'
        print("Callback: kEdsObjectEvent_DirItemCreated received; valid file reference obtained.")
    return 0  # Return EDS_ERR_OK

# Create a callback instance (ensure it remains referenced).
object_event_callback = CALLBACK_FUNC(object_event_handler)



# Load the EDSDK dynamic library.
# Adjust the library name/path depending on your operating system.
# Load the appropriate library depending on the operating system.
if sys.platform == 'win32':
    edsdk = ctypes.WinDLL('EDSDK.dll')
elif sys.platform == 'darwin':
    # Adjust the path if necessary; this assumes the EDSDK framework is installed.
    edsdk = ctypes.CDLL('EDSDK_v13.19.0_Macintosh/EDSDK/Framework/EDSDK.framework/EDSDK')
else:
    edsdk = ctypes.CDLL('libEDSDK.so')
# --- Function Prototypes from the EDSDK ---
edsdk.EdsInitializeSDK.restype = EdsError
edsdk.EdsTerminateSDK.restype = EdsError

edsdk.EdsGetCameraList.restype = EdsError
edsdk.EdsGetCameraList.argtypes = [ctypes.POINTER(EdsCameraListRef)]

edsdk.EdsGetChildCount.restype = EdsError
edsdk.EdsGetChildCount.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_uint32)]

edsdk.EdsGetChildAtIndex.restype = EdsError
edsdk.EdsGetChildAtIndex.argtypes = [ctypes.c_void_p, ctypes.c_int, ctypes.POINTER(EdsCameraRef)]

edsdk.EdsOpenSession.restype = EdsError
edsdk.EdsOpenSession.argtypes = [EdsCameraRef]

edsdk.EdsCloseSession.restype = EdsError
edsdk.EdsCloseSession.argtypes = [EdsCameraRef]

edsdk.EdsSendCommand.restype = EdsError
edsdk.EdsSendCommand.argtypes = [EdsCameraRef, ctypes.c_uint32, ctypes.c_uint32]

edsdk.EdsSetPropertyData.restype = EdsError
edsdk.EdsSetPropertyData.argtypes = [EdsCameraRef, ctypes.c_uint32, ctypes.c_int, ctypes.c_uint32, ctypes.c_void_p]

edsdk.EdsSetObjectEventHandler.restype = EdsError
edsdk.EdsSetObjectEventHandler.argtypes = [EdsCameraRef, ctypes.c_uint32, CALLBACK_FUNC, ctypes.c_void_p]

edsdk.EdsRelease.restype = ctypes.c_uint32
edsdk.EdsRelease.argtypes = [ctypes.c_void_p]

edsdk.EdsCreateMemoryStream.restype = EdsError
edsdk.EdsCreateMemoryStream.argtypes = [ctypes.c_uint32, ctypes.POINTER(EdsStreamRef)]

edsdk.EdsDownload.restype = EdsError
edsdk.EdsDownload.argtypes = [ctypes.c_void_p, ctypes.c_uint32, EdsStreamRef]

edsdk.EdsDownloadComplete.restype = EdsError
edsdk.EdsDownloadComplete.argtypes = [ctypes.c_void_p]

edsdk.EdsGetPointer.restype = EdsError
edsdk.EdsGetPointer.argtypes = [EdsStreamRef, ctypes.POINTER(ctypes.c_void_p)]

edsdk.EdsGetLength.restype = EdsError
edsdk.EdsGetLength.argtypes = [EdsStreamRef, ctypes.POINTER(ctypes.c_uint32)]

edsdk.EdsGetDirectoryItemInfo.restype = EdsError
edsdk.EdsGetDirectoryItemInfo.argtypes = [ctypes.c_void_p, ctypes.POINTER(EdsDirectoryItemInfo)]

# --- Helper for Error Checking ---
def check_error(err):
    if err != 0:
        raise Exception(f"EDSDK error: 0x{err:08x}")

# --- Full Implementation ---
def main():
    global captured_file_ref
    # 1. Initialize the SDK.
    err = edsdk.EdsInitializeSDK()
    check_error(err)
    print("SDK Initialized.")
    
    try:
        # 2. Get the camera list.
        camera_list = EdsCameraListRef()
        err = edsdk.EdsGetCameraList(ctypes.byref(camera_list))
        check_error(err)
        print("Camera list obtained.")
        
        # 3. Get number of connected cameras.
        count = ctypes.c_uint32(0)
        err = edsdk.EdsGetChildCount(camera_list, ctypes.byref(count))
        check_error(err)
        print(f"Number of cameras found: {count.value}")
        if count.value == 0:
            raise Exception("No cameras connected.")
        
        # 4. Get reference to the first camera.
        camera = EdsCameraRef()
        err = edsdk.EdsGetChildAtIndex(camera_list, 0, ctypes.byref(camera))
        check_error(err)
        print("Camera reference acquired.")
        
        # 5. Open a session with the camera.
        err = edsdk.EdsOpenSession(camera)
        check_error(err)
        print("Camera session opened.")
        
        # 6. Register the object event handler.
        #    Use the event mask that registers for all events.
        err = edsdk.EdsSetObjectEventHandler(camera, kEdsObjectEvent_All, object_event_callback, None)
        check_error(err)
        print("Object event handler registered.")
        
        # 7. Set the camera's SaveTo property to host.
        save_to_value = ctypes.c_uint32(kEdsSaveTo_Host)
        err = edsdk.EdsSetPropertyData(camera, kEdsPropID_SaveTo, 0,
                                       ctypes.sizeof(save_to_value),
                                       ctypes.byref(save_to_value))
        check_error(err)
        print("Camera set to save images to host.")
        
        # 8. Trigger the capture command.
        err = edsdk.EdsSendCommand(camera, kEdsCameraCommand_TakePicture, 0)
        check_error(err)
        print("Capture command sent; camera is taking a picture.")
        
        # 9. Wait (with timeout) for the event callback to provide a valid file reference.
        timeout = 15  # seconds (adjust if needed)
        interval = 0.5
        waited = 0.0
        while captured_file_ref is None and waited < timeout:
            time.sleep(interval)
            waited += interval
        
        if captured_file_ref is None:
            raise Exception("Timed out waiting for image capture event.")
        print("Image capture event received.")
        
        # 10. Retrieve file information.
        dir_info = EdsDirectoryItemInfo()
        err = edsdk.EdsGetDirectoryItemInfo(captured_file_ref, ctypes.byref(dir_info))
        check_error(err)
        file_size = dir_info.size
        file_name = dir_info.szFileName.decode('utf-8', errors='ignore')
        print(f"File Info: Name = {file_name}, Size = {file_size} bytes.")
        
        # 11. Create a memory stream for download.
        mem_stream = EdsStreamRef()
        buffer_size = file_size + 1024  # Buffer slightly larger than file size.
        err = edsdk.EdsCreateMemoryStream(buffer_size, ctypes.byref(mem_stream))
        check_error(err)
        print("Memory stream created for download.")
        
        # 12. Download the image into the memory stream.
        err = edsdk.EdsDownload(captured_file_ref, file_size, mem_stream)
        check_error(err)
        err = edsdk.EdsDownloadComplete(captured_file_ref)
        check_error(err)
        print("Download complete.")
        
        # 13. Retrieve the pointer and actual length from the memory stream.
        data_ptr = ctypes.c_void_p()
        err = edsdk.EdsGetPointer(mem_stream, ctypes.byref(data_ptr))
        check_error(err)
        actual_length = ctypes.c_uint32(0)
        err = edsdk.EdsGetLength(mem_stream, ctypes.byref(actual_length))
        check_error(err)
        print(f"Downloaded data length: {actual_length.value} bytes.")
        
        # 14. Convert the memory block to a Python bytes object.
        buffer = (ctypes.c_char * actual_length.value).from_address(data_ptr.value)
        image_data = bytes(buffer)
        print("Image data obtained in memory.")
        
        # (Optional) Save the image data to disk for verification.
        with open("captured_image.jpg", "wb") as f:
            f.write(image_data)
        print("Image saved to 'captured_image.jpg'.")
        
        # 15. Release SDK objects.
        edsdk.EdsRelease(mem_stream)
        edsdk.EdsRelease(captured_file_ref)
        edsdk.EdsRelease(camera)
        edsdk.EdsRelease(camera_list)
        print("Camera objects released.")
        
    finally:
        err = edsdk.EdsTerminateSDK()
        check_error(err)
        print("SDK terminated.")

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("Error:", e)


SDK Initialized.
Camera list obtained.
Number of cameras found: 1
Camera reference acquired.
Camera session opened.
